# Testing Misc Methods

## Dataloader with augmentation test

In [4]:
import os
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image
import imageio

import torch
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.utils import make_grid, save_image
import torch.nn as nn

from imgaug import parameters as iap
from imgaug import augmenters as iaa
import imgaug as ia


%matplotlib inline

# Custom includes
from dataloaders.alphapilot import AlphaPilotSegmentation
from dataloaders import utils

class Args():
    input_images_path = 'data/dataset/train/images'
    label_images_path = 'data/dataset/train/labels'

args = Args()
testBatchSize = 16
results_store_dir = 'data/test-aug'
imsize = 512

augs_test = iaa.Sequential([
    # Geometric Augs
    iaa.Resize((imsize, imsize), interpolation='cubic'),
#     iaa.Fliplr(0.5),
#     iaa.Flipud(0.5),
#     iaa.Rot90((0, 4)),
    
    # Blur and Noise
#     iaa.Sometimes(0.15, iaa.OneOf([iaa.GaussianBlur(sigma=(1.5, 2.5), name="gaus-blur"),
#                                    iaa.MotionBlur(k=13, angle=[0, 180, 270, 360], direction=[-1.0, 1.0],
#                                                  name='motion-blur'),
#                                   ])),

    # Color, Contrast, etc
#     iaa.Sometimes(0.5, iaa.CoarseDropout(0.05, size_px=(2,4), per_channel=1.0, min_size=2, name='dropout')),
#     iaa.SomeOf((0, None), [ iaa.Sometimes(0.5, iaa.GammaContrast((0.5, 1.5), name="contrast")),    
#                             iaa.Sometimes(0.5, iaa.Multiply((0.40, 1.60), per_channel=1.0, name="brightness")),
#                             iaa.Sometimes(0.5, iaa.AddToHueAndSaturation((-30, 30), name="hue-sat")),
#                           ]),
    
    # Affine
    iaa.Sometimes(0.99, iaa.Affine(scale={"x": (0.3, 0.7), "y": 1.0})),
    iaa.Sometimes(0.99, iaa.Affine(scale=(0.3, 0.7))),
])

db_test = AlphaPilotSegmentation(
    input_dir=args.input_images_path, label_dir=args.label_images_path,
    transform=augs_test,
    input_only=["gaus-blur", "motion-blur", "brightness", "contrast",
                "hue-sat", "dropout"]
)
testloader = DataLoader(db_test, batch_size=testBatchSize, shuffle=False, num_workers=4, drop_last=True)

for ii, sample_batched in enumerate(testloader):
    inputs, labels, sample_filename = sample_batched
    print(sample_filename)
    
    labels_colormap = utils.decode_seg_map_sequence(labels.squeeze(1).numpy()).type(torch.FloatTensor)    
    
#     sample = torch.cat((inputs, labels_colormap), 0)
#     img_grid = make_grid(sample, nrow=4, padding=2)

    imgs_per_row = 8
    images = []
    for i in range (0, testBatchSize):
        images.append(inputs[i])
        images.append(labels_colormap[i])
    
    img_grid = make_grid(images, nrow=imgs_per_row, padding=2)
    
    save_image(img_grid, os.path.join(results_store_dir, sample_filename[0] + '-results.png'))


('IMG_1001-rgb', 'IMG_1003-rgb', 'IMG_1004-rgb', 'IMG_1008-rgb', 'IMG_1116-rgb', 'IMG_1122-rgb', 'IMG_1123-rgb', 'IMG_1124-rgb', 'IMG_1125-rgb', 'IMG_1126-rgb', 'IMG_1127-rgb', 'IMG_1128-rgb', 'IMG_1129-rgb', 'IMG_1134-rgb', 'IMG_1140-rgb', 'IMG_1141-rgb')
('IMG_1143-rgb', 'IMG_1145-rgb', 'IMG_1146-rgb', 'IMG_1148-rgb', 'IMG_1152-rgb', 'IMG_1154-rgb', 'IMG_1155-rgb', 'IMG_1156-rgb', 'IMG_1158-rgb', 'IMG_1159-rgb', 'IMG_1160-rgb', 'IMG_1162-rgb', 'IMG_1165-rgb', 'IMG_1169-rgb', 'IMG_1181-rgb', 'IMG_1186-rgb')
('IMG_1187-rgb', 'IMG_1188-rgb', 'IMG_1191-rgb', 'IMG_1193-rgb', 'IMG_1196-rgb', 'IMG_1199-rgb', 'IMG_1200-rgb', 'IMG_1201-rgb', 'IMG_1203-rgb', 'IMG_1206-rgb', 'IMG_1209-rgb', 'IMG_1211-rgb', 'IMG_1212-rgb', 'IMG_1213-rgb', 'IMG_1214-rgb', 'IMG_1215-rgb')
('IMG_1217-rgb', 'IMG_1219-rgb', 'IMG_1220-rgb', 'IMG_1221-rgb', 'IMG_1223-rgb', 'IMG_1226-rgb', 'IMG_1227-rgb', 'IMG_1231-rgb', 'IMG_1234-rgb', 'IMG_1236-rgb', 'IMG_1238-rgb', 'IMG_1239-rgb', 'IMG_1240-rgb', 'IMG_1241-rgb', 'IMG

('IMG_2358-rgb', 'IMG_2360-rgb', 'IMG_2361-rgb', 'IMG_2367-rgb', 'IMG_2371-rgb', 'IMG_2372-rgb', 'IMG_2373-rgb', 'IMG_2379-rgb', 'IMG_2380-rgb', 'IMG_2383-rgb', 'IMG_2384-rgb', 'IMG_2388-rgb', 'IMG_2389-rgb', 'IMG_2390-rgb', 'IMG_2393-rgb', 'IMG_2394-rgb')
('IMG_2398-rgb', 'IMG_2399-rgb', 'IMG_2402-rgb', 'IMG_2405-rgb', 'IMG_2407-rgb', 'IMG_2408-rgb', 'IMG_2409-rgb', 'IMG_2410-rgb', 'IMG_2411-rgb', 'IMG_2412-rgb', 'IMG_2416-rgb', 'IMG_2420-rgb', 'IMG_2422-rgb', 'IMG_2423-rgb', 'IMG_2426-rgb', 'IMG_2428-rgb')
('IMG_2429-rgb', 'IMG_2433-rgb', 'IMG_2436-rgb', 'IMG_2443-rgb', 'IMG_2446-rgb', 'IMG_2447-rgb', 'IMG_2448-rgb', 'IMG_2451-rgb', 'IMG_2453-rgb', 'IMG_2456-rgb', 'IMG_2461-rgb', 'IMG_2465-rgb', 'IMG_2470-rgb', 'IMG_2471-rgb', 'IMG_2472-rgb', 'IMG_2474-rgb')
('IMG_2481-rgb', 'IMG_2482-rgb', 'IMG_2483-rgb', 'IMG_2484-rgb', 'IMG_2485-rgb', 'IMG_2487-rgb', 'IMG_2488-rgb', 'IMG_2490-rgb', 'IMG_2491-rgb', 'IMG_2495-rgb', 'IMG_2497-rgb', 'IMG_2504-rgb', 'IMG_2505-rgb', 'IMG_2513-rgb', 'IMG

KeyboardInterrupt: 

# Inference AlphaPilot Test

In [10]:
import json
from pprint import pprint
import glob
import cv2
import numpy as np
from random import shuffle
import time
from matplotlib import pyplot as plt
%matplotlib inline

from inference_alphapilot import inferenceAlphaPilot


folder = 'data/dataset/test/images/'
img_file = glob.glob(folder + '*.JPG')
img_keys = [img_i.split('/')[-1] for img_i in img_file]

inference = inferenceAlphaPilot(checkpoint_path = 'checkpoint/checkpoint.pth')

time_all = []
pred_dict = {}
for img_key in img_keys:
    img =cv2.imread(folder+img_key)
    img =cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    tic = time.monotonic()

    # Run inference
    mask = inference.inferenceOnNumpy(img)
    
#     # Resize Mask
    h, w = img.shape[:2]
    resized_mask = cv2.resize(mask, (w, h))
    print('resized:', resized_mask.shape)
    
    toc = time.monotonic()
    pred_dict[img_key] = resized_mask
    time_all.append(toc-tic)

mean_time = np.mean(time_all)
ci_time = 1.96*np.std(time_all)
freq = np.round(1/mean_time,2)
    
print('95% confidence interval for inference time is {0:.2f} +/- {1:.4f}.'.format(mean_time,ci_time))
print('Operating frequency from loading image to getting results is {0:.2f}.'.format(freq))






Constructing DeepLabv3+ model...
Number of classes: 2
Output stride: 8
Number of Input Channels: 3


/home/shrek/anaconda3/envs/pytorch/lib/python3.6/site-packages/torch/nn/functional.py:2351: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")


resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864

resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864

resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864, 1296)
resized: (864

In [9]:
import cv2
import numpy as np
import time

folder = 'data/dataset/test/images/'
img_file = glob.glob(folder + '*.JPG')
img_keys = [img_i.split('/')[-1] for img_i in img_file]

time_all = []
pred_dict = {}
tic1 = time.monotonic()
for img_key in img_keys:
    img =cv2.imread(folder+img_key)
    img =cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    tic = time.monotonic()

    h, w = img.shape[:2]
    mask = cv2.resize(img, (512, 512))
    print(mask.shape)
    resized_mask = cv2.resize(mask, (w, h))
    print(resized_mask.shape)
    
    toc = time.monotonic()
    pred_dict[img_key] = resized_mask
    time_all.append(toc-tic)

mean_time = np.mean(time_all)
ci_time = 1.96*np.std(time_all)
freq = np.round(1/mean_time,2)

toc1 = time.monotonic()
print('Total time taken:', toc1-tic1)

print('95% confidence interval for inference time is {0:.2f} +/- {1:.4f}.'.format(mean_time,ci_time))
print('Operating frequency from loading image to getting results is {0:.2f}.'.format(freq))

(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)


(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)


(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)


(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
(864, 1296, 3)
(512, 512, 3)
